<a href="https://colab.research.google.com/github/zahraDehghanian97/Apriori/blob/master/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load data

In [1]:
!wget https://github.com/zahraDehghanian97/Apriori/blob/master/records.txt

--2022-12-07 06:16:33--  https://github.com/zahraDehghanian97/Apriori/blob/master/records.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘records.txt’

records.txt             [ <=>                ] 136.22K  --.-KB/s    in 0.01s   

2022-12-07 06:16:33 (13.4 MB/s) - ‘records.txt’ saved [139489]



In [2]:
import pandas as pd
items = []
extended = []
with open("records.txt") as file:
    for line in file:
        s = line.split(" ")
        items.append(s)
        extended.extend(s)
print(len(items))
unique_items = list(set(extended))
print(len(unique_items))

1546
2839


<h2>Convert Items to Number</h2>

In [3]:
dictionaries =dict(zip(tuple(unique_items), tuple(range(len(unique_items)))))
numbered_items = []
for i in range(len(items)):
  temp = []
  for j in items[i] :
    temp.append(dictionaries[j])
  numbered_items.append(list(set(temp)))

print(numbered_items[:10])


[[1813], [1813], [1813], [1813], [1813], [1813], [1813], [1813], [1125, 678], [932, 2725, 1352, 1584, 1683, 604]]


<h2>Change Categorical data into numbered</h2>

In [4]:
dict_data = {'items': numbered_items}
df = pd.DataFrame.from_dict(dict_data)
items = numbered_items
df

,items
0,[1813]
1,[1813]
2,[1813]
3,[1813]
4,[1813]
...,...
1541,[1813]
1542,"[0, 706, 615, 2702, 2095, 2814]"
1543,"[0, 660]"
1544,[1211]


<h2>First Frequent Itemset (L1)</h2>

In [5]:
def count_support(database_dataframe, candidate_list):
    count_candidate = []
    #Set the Initial value of Candidate
    for i in range(len(candidate_list)):
        count_candidate.append((candidate_list[i], 0))
    df_candidate = pd.DataFrame(count_candidate, columns=['itemset', 'sup'])
    for i in range(len(database_dataframe)):
        for j in range(len(count_candidate)):
            if (set(df_candidate['itemset'][j])).issubset(set(database_dataframe['items'][i])): 
                df_candidate.loc[j, 'sup'] += 1
    return df_candidate

In [6]:
candidate1 = []
for i in range(len(unique_items)): candidate1.append([i])
count_candidate1_df = count_support(df,candidate1 )

In [7]:
def filter_sup(candidate):
    minimum_sup = 25
    filtering = candidate['sup'] > minimum_sup
    freq = candidate[filtering]
    return freq

In [8]:
freq_itemset1 = filter_sup(count_candidate1_df)
freq_itemset1_reset = freq_itemset1.reset_index(drop=True)
freq_itemset1=freq_itemset1_reset
freq_itemset1

,itemset,sup
0,[0],1001
1,[107],48
2,[119],37
3,[152],43
4,[233],27
5,[247],30
6,[335],56
7,[354],62
8,[383],95
9,[457],51


# Second Frequent Itemset (L2)


In [9]:
import numpy
def self_join(prev_freq_itemset):
    self_join_candidate = []
    for i in range(len(prev_freq_itemset['itemset'])):
        for j in range((i+1), len(prev_freq_itemset['itemset'])):
            itemset_i = prev_freq_itemset['itemset'][i]
            itemset_j = prev_freq_itemset['itemset'][j]
            if(type(itemset_i) == numpy.int64 and type(itemset_j) == numpy.int64):
                itemset_i = {itemset_i}
                itemset_j = {itemset_j}
            union_candidate = itemset_i.union(itemset_j)

            if union_candidate not in self_join_candidate:
                self_join_candidate.append(union_candidate)
    return self_join_candidate

In [10]:
import numpy
def join(itemset1,itemset2):
    join_candidate = []
    for i in range(len(itemset1['itemset'])):
        for j in range(len(itemset2['itemset'])):
            itemset_i = set(itemset1['itemset'][i])
            itemset_j = set(itemset2['itemset'][j])
            union_candidate = itemset_i.union(itemset_j)
            if ((len(list(union_candidate))== (len(list(itemset_i))+len(list(itemset_j)))) and (union_candidate not in join_candidate)):
                join_candidate.append(list(union_candidate))
    return join_candidate

In [44]:
candidate2_list = join(freq_itemset1,freq_itemset1)
candidate2_list

[[0, 107],
 [0],
 [0, 152],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0, 816],
 [0],
 [0],
 [0],
 [0, 1147],
 [0],
 [0],
 [0, 1354],
 [0],
 [0],
 [0],
 [0, 1624],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0, 2272],
 [0],
 [0],
 [0],
 [0],
 [0, 2569],
 [0],
 [0],
 [0],
 [0],
 [0, 107],
 [107, 119],
 [107],
 [233, 107],
 [107, 247],
 [107, 335],
 [354, 107],
 [107, 383],
 [457, 107],
 [107, 557],
 [562, 107],
 [611, 107],
 [107, 751],
 [107, 783],
 [785, 107],
 [107],
 [107, 823],
 [107, 950],
 [1002, 107],
 [107],
 [1152, 107],
 [1265, 107],
 [1354, 107],
 [1394, 107],
 [1416, 107],
 [107, 1556],
 [1624, 107],
 [1688, 107],
 [107, 1707],
 [1728, 107],
 [107, 1813],
 [107, 1933],
 [107, 2071],
 [2272, 107],
 [107, 2294],
 [107, 2430],
 [107, 2503],
 [2512, 107],
 [2569, 107],
 [2603, 107],
 [107, 2702],
 [107, 2751],
 [107, 2774],
 [0],
 [107, 119],
 [152, 119],
 [119],
 [119],
 [335, 119],
 [354, 119],
 [383, 119],
 [457, 119],
 [119],
 [562, 119],
 [119],
 [

In [43]:
count_candidate2_df = count_support(df, candidate2_list)
freq_itemset2 = filter_sup(count_candidate2_df)
freq_itemset2 = freq_itemset2.reset_index(drop=True)
freq_itemset2

,itemset,sup
0,[0],1001
1,[0],1001
2,[0],1001
3,[0],1001
4,[0],1001
...,...,...
349,"[2774, 823]",61
350,"[1265, 2774]",61
351,"[1556, 2774]",63
352,"[1933, 2774]",55


<h2>Third Frequent Itemset (L3)</h2>

In [13]:
join_result = join(freq_itemset2,freq_itemset1)
print('join result : ')
print(join_result)

join result : 
[[0, 107, 119], [0, 152, 119], [0, 233, 119], [0, 247, 119], [0, 335, 119], [0, 354, 119], [0, 383, 119], [0, 457, 119], [0, 557, 119], [0, 562, 119], [0, 611, 119], [0, 751, 119], [0, 783, 119], [0, 785, 119], [0, 816, 119], [0, 823, 119], [0, 950, 119], [0, 1002, 119], [0, 1147, 119], [0, 1152, 119], [0, 1265, 119], [0, 1354, 119], [0, 1394, 119], [0, 1416, 119], [0, 1556, 119], [0, 1624, 119], [0, 1688, 119], [0, 1707, 119], [0, 1728, 119], [0, 1813, 119], [0, 1933, 119], [0, 2071, 119], [0, 2272, 119], [0, 2294, 119], [0, 2430, 119], [0, 2503, 119], [0, 2512, 119], [0, 2569, 119], [0, 2603, 119], [0, 2702, 119], [0, 2751, 119], [0, 2774, 119], [0, 233, 107], [0, 233, 119], [0, 233, 152], [0, 233, 247], [0, 233, 335], [0, 233, 354], [0, 233, 383], [0, 233, 457], [0, 233, 557], [0, 233, 562], [0, 233, 611], [0, 233, 751], [0, 233, 783], [0, 233, 785], [0, 233, 816], [0, 233, 823], [0, 233, 950], [0, 233, 1002], [0, 233, 1147], [0, 233, 1152], [0, 233, 1265], [0, 233, 1

## Pruning

In [14]:
def get_subset(candidate):
    temp = []
    final = []
    for i in range(len(candidate)):
        for j in range(len(candidate)):
            if i != j:
                temp.append(candidate[j])
        temp_set = set(temp)
        final.append(temp_set)
        temp.clear()
    print('Subset from {} : {}'.format(candidate, final))
    return final

def pruning(candidate_set, freq_itemset):
    
    temp = []
    for idx, value in enumerate(candidate_set):
        print("---------------------------------------------------")
        print('Candidate to evaluate : ', value)
        list_candidate = list(value)
        temp_candidate = (get_subset(list_candidate))
        checks = True
        for temp_item in temp_candidate:
            c = False
            for i in freq_itemset['itemset'] :
              if temp_item == i:
                c = True
            print('\nCheck {} candidate in Previous Frequent Itemset result : {} '.format(temp_item , c))
            checks = checks and c    
        if checks : 
            print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
            print('\nAll of {} subset contained in \n{}'.format(candidate_set, freq_itemset))
            if value not in temp:
                temp.append(value)
    return temp

In [15]:
candidate3_list = pruning(join_result, freq_itemset2)
print("final candidate 3 list : ")
candidate3_list

Streaming output truncated to the last 5000 lines.

Check candidate in Previous Frequent Itemset
 False
Temp item {2774, 2430}

Check candidate in Previous Frequent Itemset
 False
Subset from [2503, 2774, 335] : [{2774, 335}, {335, 2503}, {2774, 2503}]
Temp item {2774, 335}

Check candidate in Previous Frequent Itemset
 False
Temp item {335, 2503}

Check candidate in Previous Frequent Itemset
 False
Temp item {2774, 2503}

Check candidate in Previous Frequent Itemset
 False
Subset from [2512, 2774, 335] : [{2774, 335}, {2512, 335}, {2512, 2774}]
Temp item {2774, 335}

Check candidate in Previous Frequent Itemset
 False
Temp item {2512, 335}

Check candidate in Previous Frequent Itemset
 False
Temp item {2512, 2774}

Check candidate in Previous Frequent Itemset
 False
Subset from [2569, 2774, 335] : [{2774, 335}, {2569, 335}, {2569, 2774}]
Temp item {2774, 335}

Check candidate in Previous Frequent Itemset
 False
Temp item {2569, 335}

Check candidate in Previous Frequent Itemset
 False

[]

In [16]:
count_candidate3_df = count_support(df, candidate3_list)
freq_itemset3 = filter_sup(count_candidate3_df)
freq_itemset3

,itemset,sup


## All Frequent Itemset

In [17]:
frequent_itemset = pd.concat([freq_itemset1, freq_itemset2, freq_itemset3], axis=0)
frequent_itemset_final = frequent_itemset.reset_index(drop=True)
frequent_itemset_final

,itemset,sup
0,[0],1001
1,[107],48
2,[119],37
3,[152],43
4,[233],27
...,...,...
393,"[2774, 823]",61
394,"[1265, 2774]",61
395,"[1556, 2774]",63
396,"[1933, 2774]",55


# Find Association Rules

In [18]:
def get_subset_list(candidate):
    temp = []
    final = []
    for i in range(len(candidate)):
        for j in range(len(candidate)):
            if i != j:
                temp.append(candidate[j])
        temp_set = list(set(temp))
        final.append(temp_set)
        temp.clear()
    print('Subset from {} : {}'.format(candidate, final))
    return final

In [42]:
freq_itemset2

,itemset,sup
0,[0],1001
1,[0],1001
2,[0],1001
3,[0],1001
4,[0],1001
...,...,...
349,"[2774, 823]",61
350,"[1265, 2774]",61
351,"[1556, 2774]",63
352,"[1933, 2774]",55


In [40]:
# for idx, value in enumerate(freq_itemset3):
#     list_candidate = list(value)
#     temp_candidate = (get_subset(list_candidate))
#     df_candidate = freq_itemset2[freq_itemset2['itemset']==temp_candidate]
#     min_support_itemset =df_candidate[df_candidate['sup']==min(df_candidate['sup'])]
#     print(str(min_support_itemset['itemset'])+" ->"+(str(list_candidate.remove(list(min_support_itemset['itemset']))))+" , confidence score : "+str(value['sup']/min_support_itemset['sup']))

for itemset in freq_itemset2.values : 
    list_candidate = itemset[0]
    print(list_candidate)   
    temp_candidate = (get_subset_list(list_candidate))
         
    df_candidate=freq_itemset1[freq_itemset1.isin((temp_candidate)).any(1)]
    min_support_itemset =df_candidate[df_candidate['sup']==min(df_candidate['sup'])]
    print("+++++++++++++++++++++++++++")
    print(min_support_itemset['itemset'])
    print(list_candidate.remove((min_support_itemset['itemset'].item()[0])))
    # print(str(min_support_itemset['itemset'])+" ->"+(str(list_candidate.remove((min_support_itemset['itemset'].item()[0])))))
    print(itemset[1])
    print(min(df_candidate['sup']))
    # print(" confidence score : "+str(itemset[1]/min(df_candidate['sup'])))


[0]
Subset from [0] : [[]]


ValueError: ignored